In [ ]:
# # install.packages("remotes")
# remotes::install_github("Thell/RcppMP")

In [1]:
library(ggplot2)
library(viridis)
library(grid)
library(gridExtra)
library("IRdisplay")
library(tidyverse)

Loading required package: viridisLite

Warning message:
“Your system is mis-configured: ‘/var/db/timezone/localtime’ is not a symlink”
Warning message:
“‘/var/db/timezone/localtime’ is not identical to any known timezone file”
Warning message:
“Failed to locate timezone database”
── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ lubridate 1.9.3     ✔ tibble    3.2.1
✔ purrr     1.0.2     ✔ tidyr     1.3.1
── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::combine() masks gridExtra::combine()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [ ]:
library(devtools)

#remove.packages("ptdalgorithms")
# #devtools::install_github("kaspermunch/PtDAlgorithms")
# devtools::install_local('../PtDAlgorithms', quiet=FALSE)
#devtools::install_local()
devtools::load_all(path="./", quiet=FALSE, recompile=TRUE)

library(ptdalgorithms)

## Rebuild Rcpp interface

In [14]:
Rcpp::compileAttributes() 

## Generate documentation

In [19]:
install.packages("roxygen2md")

also installing the dependency ‘rex’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [17]:
library(roxygen2)
Rcpp::compileAttributes()           # this updates the Rcpp layer from C++ to R
roxygen2::roxygenize(roclets="rd")  # this updates the documentation based on roxygen comments

ℹ Loading ptdalgorithms


In [20]:
roxygen2md::roxygen2md(scope = "none")          # "none" only adds list(markdown = TRUE) to the Roxygen field in DESCRIPTION

✔ Setting active project to "/Users/kmt/google_drive/projects/phasestype/PtDAlgorithms".

ℹ Running `devtools::document()`
ℹ Updating ptdalgorithms documentation
ℹ Loading ptdalgorithms
✖ Skipping NAMESPACE
ℹ It already exists and was not generated by roxygen2.
• Review the changes carefully
• Commit the changes to version control
• Run `roxygen2md::roxygen2md()` with a stricter `scope` argument
✔ Setting active project to "<no active project>".



In [21]:
roxygen2md::roxygen2md(scope = "simple")        # "simple" only converts elements like \code{} and \emph{}

✔ Setting active project to "/Users/kmt/google_drive/projects/phasestype/PtDAlgorithms".

ℹ Running `devtools::document()`
ℹ Updating ptdalgorithms documentation
ℹ Loading ptdalgorithms
✖ Skipping NAMESPACE
ℹ It already exists and was not generated by roxygen2.
• Review the changes carefully
• Commit the changes to version control
• Run `roxygen2md::roxygen2md()` with a stricter `scope` argument
✔ Setting active project to "<no active project>".



In [22]:
roxygen2md::roxygen2md(scope = "full")          # "full" runs all conversions

✔ Setting active project to "/Users/kmt/google_drive/projects/phasestype/PtDAlgorithms".

ℹ Running `devtools::document()`
ℹ Updating ptdalgorithms documentation
ℹ Loading ptdalgorithms
✖ Skipping NAMESPACE
ℹ It already exists and was not generated by roxygen2.
• Review the changes carefully
• Commit the changes to version control
✔ Setting active project to "<no active project>".



## Testing

In [23]:
source("./utils/plotting.R")

theme_set(theme_bw())
despine <- theme(panel.border = element_blank(), panel.grid.major = element_blank(),
panel.grid.minor = element_blank(), axis.line = element_line(colour = "black"),
                # text=element_text(family="Arial")
                )

In [24]:
standard_coalescent <- function(n, size, graph=NULL, edge_trans=NULL, epoque=0) {

    state_vector_length <- n + 1

    if (is.null(graph)) {
    
        graph <- create_graph(state_vector_length)
        starting_vertex <- vertex_at(graph, 1)
        initial_state <- c(rep(0, n), epoque)
        initial_state[1] <- n
        
        add_edge(
          starting_vertex,
          create_vertex(graph, initial_state),
          1
        )
        index <- 2
        
    } else {
        nr_states <- vertices_length(graph)
        for (i in 2:(nr_states-1)) {
            if (is.nan(edge_trans[i-1])) {
                next;
            }
            vertex <- vertex_at(graph, i)
            state <- vertex$state
            sister_state <- state
            sister_state[length(sister_state)] <- epoque
            add_edge(
              vertex,
              find_or_create_vertex(graph, sister_state),
              edge_trans[i-1], c(edge_trans[i-1]) # -1 becuase first index is 2 and first element of edge_trans is 1
            )
        }
        index <- nr_states + 1
    }

    while (index <= vertices_length(graph)) {
      vertex <- vertex_at(graph, index)

        # loop over all classes of lineages
      for (i in 1:n) {
        for (j in i:n) {
          state <- vertex$state

          # if same class, there need to be at least two to coalesce
          if (i == j) {
            if (state[i] < 2) {
              next; # an absorbing state that is not the last state
            }
            # coal rate
            rate <- state[i] * (state[i] - 1) / 2
          } else {
            # else at least one in each class to coalesce
            if (state[i] < 1 || state[j] < 1) {
              next;
            }
            # number of combinations
            rate <- state[i] * state[j]
          }

          rate = rate / size
            
          # copy state
          child_state <- state
            
          # update child state
          child_state[i] <- child_state[i] - 1
          child_state[j] <- child_state[j] - 1
          child_state[i+j] <- child_state[i+j] + 1

          add_edge(
              vertex,
              find_or_create_vertex(graph, child_state),
              rate, c(rate)
            )
        }
      }
        
      index <- index + 1
    }
    if (epoque == 0) {
        return(graph)
    }
}

r_add_epoque <- function(n, size, graph, epoque_trans, epoque)
    return(standard_coalescent(n, size, graph, epoque_trans, epoque))

In [25]:
n_samples <- 4

In [26]:
graph <- standard_coalescent(n_samples, 1)
graph_as_matrix(graph)

$states
     [,1] [,2] [,3] [,4] [,5]
[1,]    4    0    0    0    0
[2,]    2    1    0    0    0
[3,]    0    2    0    0    0
[4,]    1    0    1    0    0

$SIM
     [,1] [,2] [,3] [,4]
[1,]   -6    6    0    0
[2,]    0   -3    1    2
[3,]    0    0   -1    0
[4,]    0    0    0   -1

$IPV
[1] 1 0 0 0

$indices
[1] 2 3 4 5

In [27]:
expected_waiting_time(graph)

[1] 1.500000 1.500000 1.333333 1.000000 1.000000 0.000000

In [28]:
expected_residence_time(graph)

[1]  0.000000  6.000000  1.333333 -2.500000 -3.333333  0.000000

In [ ]:
gam <- graph_as_matrix(graph)
plot_graph(gam, subgraphs=TRUE, rainbow=TRUE, size=c(8, 8), align=TRUE,
           fontsize=14, ranksep=1, nodesep=0.5,          
           subgraphfun=function(state) paste(state[-length(state)], collapse=""))

In [ ]:
stop_probability(graph, 1)
accumulated_visiting_time(graph, 1)

In [ ]:
x <- stop_probability(graph, 1)
stop_probs <- x[2:(length(x)-1)]
x <- accumulated_visiting_time(graph, 1)
accum_v_time <- x[2:(length(x)-1)]
epoque_trans_rates <- stop_probs / accum_v_time

add_epoque(n_samples, 5, graph, epoque_trans_rates, 1)
graph_as_matrix(graph)

In [ ]:
gam <- graph_as_matrix(graph)
plot_graph(gam, subgraphs=TRUE, rainbow=TRUE, size=c(8, 8), align=TRUE,
           fontsize=14, ranksep=1, nodesep=0.5,          
           subgraphfun=function(state) paste(state[-length(state)], collapse=""))

In [ ]:
stop_probability(graph, 1)
accumulated_visiting_time(graph, 1)

In [ ]:
x <- stop_probability(graph, 1)
stop_probs <- x[2:(length(x)-1)]
x <- accumulated_visiting_time(graph, 1)
accum_v_time <- x[2:(length(x)-1)]
epoque_trans_rates <- stop_probs / accum_v_time

add_epoque(n_samples, 10, graph, epoque_trans_rates, 2)
graph_as_matrix(graph)

In [ ]:
gam <- graph_as_matrix(graph)
plot_graph(gam, subgraphs=TRUE, rainbow=TRUE, size=c(8, 8), align=TRUE,
           fontsize=14, ranksep=1, nodesep=0.5,          
           subgraphfun=function(state) paste(state[-length(state)], collapse=""))

In [ ]:
stop_probability(graph, 1)
accumulated_visiting_time(graph, 1)

In [ ]:
graph <- standard_coalescent(n_samples, 1)
graph_as_matrix(graph)

In [ ]:
add_epoque(graph, 1/5, 1)